In [1]:
import pandas as pd
import numpy as np
np.random.seed(42)

In [2]:
df = pd.read_csv(filepath_or_buffer='./roman_numerals.txt')

In [3]:
df.head()

,ROMAN,HA
0,I,1
1,II,2
2,III,3
3,IV,4
4,V,5


In [4]:
shuffled_df = df.sample(frac=1, random_state=42)

In [5]:
roman_values = df.loc[:, 'ROMAN'].astype(np.str)
ha_values = df.loc[:, 'HA'].astype(np.str)

In [6]:
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.


In [7]:
roman_tokenizer = Tokenizer(char_level=True)
roman_tokenizer.fit_on_texts(roman_values)

ha_tokenizer = Tokenizer(char_level=True)
ha_tokenizer.fit_on_texts(ha_values)

In [8]:
roman_tokenizer.word_index, ha_tokenizer.word_index

({'I': 1, 'V': 3, 'X': 2},
 {'0': 10,
  '1': 1,
  '2': 2,
  '3': 3,
  '4': 4,
  '5': 5,
  '6': 6,
  '7': 7,
  '8': 8,
  '9': 9})

In [9]:
roman_index_to_char = {v:k for k,v in roman_tokenizer.word_index.items()}
roman_index_to_char

{1: 'I', 2: 'X', 3: 'V'}

In [10]:
ha_index_to_char = {v:k for k,v in ha_tokenizer.word_index.items()}
ha_index_to_char

{1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: '0'}

In [11]:
x_roman_1 = roman_tokenizer.texts_to_matrix(roman_values)
x_roman_2 = roman_tokenizer.texts_to_matrix(roman_values, mode='count')

In [12]:
x_roman = np.hstack([x_roman_1, x_roman_2])

In [14]:
x_roman.shape

(20, 8)

In [15]:
y_ha = ha_tokenizer.texts_to_sequences(ha_values)
y_ha = pad_sequences(y_ha, padding='post')

In [16]:
y_ha = to_categorical(y_ha)
y_ha = y_ha.reshape(20, 22)

In [17]:
y_ha

array([[ 0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,
         0.,  0.,  0.,  0.,  0.,  0., 

In [18]:
from keras.models import Sequential
from keras.layers import Dense, Activation

In [44]:
INPUT_DIM = x_roman.shape[1]
OUTPUT_DIM = y_ha.shape[1]
NUM_HIDDEN = 50

In [45]:
model = Sequential()
model.add(Dense(NUM_HIDDEN, input_dim=INPUT_DIM))
model.add(Activation('relu'))
model.add(Dense(NUM_HIDDEN))
model.add(Activation('relu'))
model.add(Dense(OUTPUT_DIM))
model.add(Activation('sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [46]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 50)                450       
_________________________________________________________________
activation_7 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 50)                2550      
_________________________________________________________________
activation_8 (Activation)    (None, 50)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 22)                1122      
_________________________________________________________________
activation_9 (Activation)    (None, 22)                0         
Total params: 4,122
Trainable params: 4,122
Non-trainable params: 0
_________________________________________________________________


In [47]:
model.fit(x=x_roman, y=y_ha, epochs=1000, validation_split=0.3)

Train on 14 samples, validate on 6 samples
Epoch 1/1000
14/14 [==============================] - 0s - loss: 0.6937 - acc: 0.5325 - val_loss: 0.6930 - val_acc: 0.5303
Epoch 2/1000
14/14 [==============================] - 0s - loss: 0.6881 - acc: 0.5519 - val_loss: 0.6859 - val_acc: 0.5379
Epoch 3/1000
14/14 [==============================] - 0s - loss: 0.6806 - acc: 0.5682 - val_loss: 0.6789 - val_acc: 0.5530
Epoch 4/1000
14/14 [==============================] - 0s - loss: 0.6732 - acc: 0.5844 - val_loss: 0.6721 - val_acc: 0.5758
Epoch 5/1000
14/14 [==============================] - 0s - loss: 0.6659 - acc: 0.6006 - val_loss: 0.6655 - val_acc: 0.5985
Epoch 6/1000
14/14 [==============================] - 0s - loss: 0.6585 - acc: 0.6234 - val_loss: 0.6589 - val_acc: 0.6061
Epoch 7/1000
14/14 [==============================] - 0s - loss: 0.6512 - acc: 0.6396 - val_loss: 0.6525 - val_acc: 0.6136
Epoch 8/1000
14/14 [==============================] - 0s - loss: 0.6437 - acc: 0.6591 - val_loss

14/14 [==============================] - 0s - loss: 0.2271 - acc: 0.9286 - val_loss: 0.3333 - val_acc: 0.8788
Epoch 67/1000
14/14 [==============================] - 0s - loss: 0.2253 - acc: 0.9286 - val_loss: 0.3336 - val_acc: 0.8788
Epoch 68/1000
14/14 [==============================] - 0s - loss: 0.2236 - acc: 0.9286 - val_loss: 0.3341 - val_acc: 0.8788
Epoch 69/1000
14/14 [==============================] - 0s - loss: 0.2220 - acc: 0.9286 - val_loss: 0.3347 - val_acc: 0.8788
Epoch 70/1000
14/14 [==============================] - 0s - loss: 0.2205 - acc: 0.9286 - val_loss: 0.3355 - val_acc: 0.8788
Epoch 71/1000
14/14 [==============================] - 0s - loss: 0.2190 - acc: 0.9286 - val_loss: 0.3363 - val_acc: 0.8788
Epoch 72/1000
14/14 [==============================] - 0s - loss: 0.2176 - acc: 0.9318 - val_loss: 0.3373 - val_acc: 0.8788
Epoch 73/1000
14/14 [==============================] - 0s - loss: 0.2162 - acc: 0.9318 - val_loss: 0.3383 - val_acc: 0.8788
Epoch 74/1000
14/14 [=

14/14 [==============================] - 0s - loss: 0.1558 - acc: 0.9481 - val_loss: 0.3861 - val_acc: 0.9242
Epoch 132/1000
14/14 [==============================] - 0s - loss: 0.1550 - acc: 0.9481 - val_loss: 0.3871 - val_acc: 0.9242
Epoch 133/1000
14/14 [==============================] - 0s - loss: 0.1542 - acc: 0.9481 - val_loss: 0.3881 - val_acc: 0.9242
Epoch 134/1000
14/14 [==============================] - 0s - loss: 0.1534 - acc: 0.9481 - val_loss: 0.3891 - val_acc: 0.9242
Epoch 135/1000
14/14 [==============================] - 0s - loss: 0.1526 - acc: 0.9481 - val_loss: 0.3900 - val_acc: 0.9242
Epoch 136/1000
14/14 [==============================] - 0s - loss: 0.1518 - acc: 0.9481 - val_loss: 0.3910 - val_acc: 0.9242
Epoch 137/1000
14/14 [==============================] - 0s - loss: 0.1510 - acc: 0.9481 - val_loss: 0.3921 - val_acc: 0.9242
Epoch 138/1000
14/14 [==============================] - 0s - loss: 0.1502 - acc: 0.9481 - val_loss: 0.3931 - val_acc: 0.9242
Epoch 139/1000


14/14 [==============================] - 0s - loss: 0.1159 - acc: 0.9513 - val_loss: 0.4539 - val_acc: 0.9318
Epoch 197/1000
14/14 [==============================] - 0s - loss: 0.1155 - acc: 0.9513 - val_loss: 0.4549 - val_acc: 0.9318
Epoch 198/1000
14/14 [==============================] - 0s - loss: 0.1150 - acc: 0.9513 - val_loss: 0.4559 - val_acc: 0.9318
Epoch 199/1000
14/14 [==============================] - 0s - loss: 0.1146 - acc: 0.9513 - val_loss: 0.4570 - val_acc: 0.9318
Epoch 200/1000
14/14 [==============================] - 0s - loss: 0.1141 - acc: 0.9513 - val_loss: 0.4580 - val_acc: 0.9318
Epoch 201/1000
14/14 [==============================] - 0s - loss: 0.1137 - acc: 0.9513 - val_loss: 0.4589 - val_acc: 0.9318
Epoch 202/1000
14/14 [==============================] - 0s - loss: 0.1132 - acc: 0.9513 - val_loss: 0.4599 - val_acc: 0.9318
Epoch 203/1000
14/14 [==============================] - 0s - loss: 0.1128 - acc: 0.9513 - val_loss: 0.4609 - val_acc: 0.9318
Epoch 204/1000


14/14 [==============================] - 0s - loss: 0.0914 - acc: 0.9643 - val_loss: 0.5096 - val_acc: 0.9318
Epoch 262/1000
14/14 [==============================] - 0s - loss: 0.0911 - acc: 0.9643 - val_loss: 0.5105 - val_acc: 0.9318
Epoch 263/1000
14/14 [==============================] - 0s - loss: 0.0907 - acc: 0.9643 - val_loss: 0.5113 - val_acc: 0.9318
Epoch 264/1000
14/14 [==============================] - 0s - loss: 0.0904 - acc: 0.9643 - val_loss: 0.5122 - val_acc: 0.9318
Epoch 265/1000
14/14 [==============================] - 0s - loss: 0.0901 - acc: 0.9643 - val_loss: 0.5130 - val_acc: 0.9318
Epoch 266/1000
14/14 [==============================] - 0s - loss: 0.0898 - acc: 0.9643 - val_loss: 0.5138 - val_acc: 0.9318
Epoch 267/1000
14/14 [==============================] - 0s - loss: 0.0895 - acc: 0.9643 - val_loss: 0.5145 - val_acc: 0.9318
Epoch 268/1000
14/14 [==============================] - 0s - loss: 0.0892 - acc: 0.9643 - val_loss: 0.5151 - val_acc: 0.9318
Epoch 269/1000


...

14/14 [==============================] - 0s - loss: 0.0334 - acc: 0.9805 - val_loss: 0.8190 - val_acc: 0.9242
Epoch 717/1000
14/14 [==============================] - 0s - loss: 0.0334 - acc: 0.9805 - val_loss: 0.8194 - val_acc: 0.9242
Epoch 718/1000
14/14 [==============================] - 0s - loss: 0.0333 - acc: 0.9805 - val_loss: 0.8197 - val_acc: 0.9242
Epoch 719/1000
14/14 [==============================] - 0s - loss: 0.0333 - acc: 0.9805 - val_loss: 0.8200 - val_acc: 0.9242
Epoch 720/1000
14/14 [==============================] - 0s - loss: 0.0333 - acc: 0.9805 - val_loss: 0.8203 - val_acc: 0.9242
Epoch 721/1000
14/14 [==============================] - 0s - loss: 0.0332 - acc: 0.9805 - val_loss: 0.8205 - val_acc: 0.9242
Epoch 722/1000
14/14 [==============================] - 0s - loss: 0.0332 - acc: 0.9805 - val_loss: 0.8209 - val_acc: 0.9242
Epoch 723/1000
14/14 [==============================] - 0s - loss: 0.0332 - acc: 0.9805 - val_loss: 0.8212 - val_acc: 0.9242
Epoch 724/1000


14/14 [==============================] - 0s - loss: 0.0317 - acc: 0.9805 - val_loss: 0.8402 - val_acc: 0.9242
Epoch 782/1000
14/14 [==============================] - 0s - loss: 0.0317 - acc: 0.9805 - val_loss: 0.8404 - val_acc: 0.9242
Epoch 783/1000
14/14 [==============================] - 0s - loss: 0.0317 - acc: 0.9805 - val_loss: 0.8406 - val_acc: 0.9242
Epoch 784/1000
14/14 [==============================] - 0s - loss: 0.0316 - acc: 0.9805 - val_loss: 0.8409 - val_acc: 0.9242
Epoch 785/1000
14/14 [==============================] - 0s - loss: 0.0316 - acc: 0.9805 - val_loss: 0.8411 - val_acc: 0.9242
Epoch 786/1000
14/14 [==============================] - 0s - loss: 0.0316 - acc: 0.9805 - val_loss: 0.8413 - val_acc: 0.9242
Epoch 787/1000
14/14 [==============================] - 0s - loss: 0.0316 - acc: 0.9805 - val_loss: 0.8416 - val_acc: 0.9242
Epoch 788/1000
14/14 [==============================] - 0s - loss: 0.0316 - acc: 0.9805 - val_loss: 0.8419 - val_acc: 0.9242
Epoch 789/1000


14/14 [==============================] - 0s - loss: 0.0305 - acc: 0.9805 - val_loss: 0.8561 - val_acc: 0.9242
Epoch 847/1000
14/14 [==============================] - 0s - loss: 0.0305 - acc: 0.9805 - val_loss: 0.8563 - val_acc: 0.9242
Epoch 848/1000
14/14 [==============================] - 0s - loss: 0.0305 - acc: 0.9805 - val_loss: 0.8565 - val_acc: 0.9242
Epoch 849/1000
14/14 [==============================] - 0s - loss: 0.0305 - acc: 0.9805 - val_loss: 0.8568 - val_acc: 0.9242
Epoch 850/1000
14/14 [==============================] - 0s - loss: 0.0305 - acc: 0.9805 - val_loss: 0.8570 - val_acc: 0.9242
Epoch 851/1000
14/14 [==============================] - 0s - loss: 0.0305 - acc: 0.9805 - val_loss: 0.8572 - val_acc: 0.9242
Epoch 852/1000
14/14 [==============================] - 0s - loss: 0.0305 - acc: 0.9805 - val_loss: 0.8575 - val_acc: 0.9242
Epoch 853/1000
14/14 [==============================] - 0s - loss: 0.0304 - acc: 0.9805 - val_loss: 0.8577 - val_acc: 0.9242
Epoch 854/1000


14/14 [==============================] - 0s - loss: 0.0297 - acc: 0.9805 - val_loss: 0.8699 - val_acc: 0.9242
Epoch 912/1000
14/14 [==============================] - 0s - loss: 0.0297 - acc: 0.9805 - val_loss: 0.8701 - val_acc: 0.9242
Epoch 913/1000
14/14 [==============================] - 0s - loss: 0.0297 - acc: 0.9805 - val_loss: 0.8703 - val_acc: 0.9242
Epoch 914/1000
14/14 [==============================] - 0s - loss: 0.0297 - acc: 0.9805 - val_loss: 0.8705 - val_acc: 0.9242
Epoch 915/1000
14/14 [==============================] - 0s - loss: 0.0297 - acc: 0.9805 - val_loss: 0.8707 - val_acc: 0.9242
Epoch 916/1000
14/14 [==============================] - 0s - loss: 0.0297 - acc: 0.9805 - val_loss: 0.8709 - val_acc: 0.9242
Epoch 917/1000
14/14 [==============================] - 0s - loss: 0.0297 - acc: 0.9805 - val_loss: 0.8711 - val_acc: 0.9242
Epoch 918/1000
14/14 [==============================] - 0s - loss: 0.0297 - acc: 0.9805 - val_loss: 0.8713 - val_acc: 0.9242
Epoch 919/1000


14/14 [==============================] - 0s - loss: 0.0291 - acc: 0.9805 - val_loss: 0.8824 - val_acc: 0.9242
Epoch 977/1000
14/14 [==============================] - 0s - loss: 0.0291 - acc: 0.9805 - val_loss: 0.8826 - val_acc: 0.9242
Epoch 978/1000
14/14 [==============================] - 0s - loss: 0.0291 - acc: 0.9805 - val_loss: 0.8828 - val_acc: 0.9242
Epoch 979/1000
14/14 [==============================] - 0s - loss: 0.0291 - acc: 0.9805 - val_loss: 0.8830 - val_acc: 0.9242
Epoch 980/1000
14/14 [==============================] - 0s - loss: 0.0291 - acc: 0.9805 - val_loss: 0.8832 - val_acc: 0.9242
Epoch 981/1000
14/14 [==============================] - 0s - loss: 0.0291 - acc: 0.9805 - val_loss: 0.8833 - val_acc: 0.9242
Epoch 982/1000
14/14 [==============================] - 0s - loss: 0.0291 - acc: 0.9805 - val_loss: 0.8835 - val_acc: 0.9242
Epoch 983/1000
14/14 [==============================] - 0s - loss: 0.0291 - acc: 0.9805 - val_loss: 0.8837 - val_acc: 0.9242
Epoch 984/1000


In [48]:
y_pred = model.predict(x_roman)

In [49]:
roman_values

0         I
1        II
2       III
3        IV
4         V
5        VI
6       VII
7      VIII
8        IX
9         X
10       XI
11      XII
12     XIII
13      XIV
14       XV
15      XVI
16     XVII
17    XVIII
18      XIX
19       XX
Name: ROMAN, dtype: object

In [50]:
y_pred.shape

(20, 22)

In [51]:
NUM_PER_CHAR = 11
results = []
for i in (range(y_pred.shape[0])):
    char1_one_hot = y_pred[i,0:11]
    char1_encoded = np.argmax(char1_one_hot)
    char1 = ''
    if char1_encoded != 0:
        char1 = ha_index_to_char[char1_encoded]    
    
    char2_one_hot = y_pred[i,11:]
    char2_encoded = np.argmax(char2_one_hot)
    char2 = ''
    if char2_encoded != 0:
        char2 = ha_index_to_char[char2_encoded]
    results.append(char1 + char2)
    

In [52]:
type(results)

list

In [53]:
type(list(roman_values))

list

In [54]:
dict(zip(list(roman_values), results))

{'I': '1',
 'II': '2',
 'III': '3',
 'IV': '6',
 'IX': '1',
 'V': '5',
 'VI': '6',
 'VII': '7',
 'VIII': '8',
 'X': '10',
 'XI': '1',
 'XII': '12',
 'XIII': '13',
 'XIV': '14',
 'XIX': '10',
 'XV': '14',
 'XVI': '14',
 'XVII': '14',
 'XVIII': '13',
 'XX': '10'}